In [1]:
from mainnet_launch.pages.gas_costs.gas_costs import fetch_tokemak_address_constants_dfs
from mainnet_launch.database.schema.postgres_operations import TableSelector, merge_tables_as_df
from mainnet_launch.data_fetching.get_events import fetch_events
from mainnet_launch.constants import ETH_CHAIN, ChainData

from mainnet_launch.database.schema.full import Transactions, Blocks, ChainlinkGasCosts
import pandas as pd
import plotly.express as px
import plotly.io as pio

pio.templates.default = None

from mainnet_launch.pages.gas_costs.gas_costs import fetch_our_gas_costs_df

df, address_constants = fetch_our_gas_costs_df()

In [ ]:
daily_gas_prices_df = df.groupby(df["datetime"].dt.date)["effective_gas_price"].apply(list).reset_index()
exploded_df = daily_gas_prices_df.explode("effective_gas_price")


hourly_gas_price_box_and_whisker_fig = px.box(
    exploded_df, x="datetime", y="effective_gas_price", title="Daily Distribution of Gas Prices"
)

hourly_gas_price_box_and_whisker_fig

AttributeError: 'DatetimeProperties' object has no attribute 'week'

In [19]:
df2 = df.set_index("datetime")

df2

,tx_hash,label,category,effective_gas_price,gas_used,gas_cost_in_eth,week_start
datetime,,,,,,,
2025-07-03 06:44:59+00:00,0x2c99190e82d5dc1dade8ccf00d22042e02c2967f5551...,Incentive Pricing (keeper_network),keeper_network,498225940,214745,0.000128,2025-07-01
2025-02-10 04:12:11+00:00,0x43d7106ccdb6e393d3f0ad61cad2e6b65c08161a1273...,Calculators (keeper_network),keeper_network,1365718489,534860,0.000877,2025-02-04
2025-05-30 12:00:59+00:00,0x1c30f836f0d75e83c0f4e9cfc755d4bc582a59b6c514...,Calculators (keeper_network),keeper_network,3357016839,1818928,0.007327,2025-05-27
2024-11-30 12:00:23+00:00,0x931aecdfe77de8c9c800968099bba0c087465787ffc8...,Calculators (keeper_network),keeper_network,7659989110,1703275,0.015656,2024-11-26
2025-01-14 04:22:47+00:00,0x3aa3206b66f420fef6efbc96699b396a807c2d52b72f...,Calculators (keeper_network),keeper_network,2841546074,414842,0.001415,2025-01-14
...,...,...,...,...,...,...,...
2025-07-15 00:07:47+00:00,0x5ca8ea61442c441f48470aabf62fc89f5cf1bb39b8b0...,Liquidator (rewards),rewards,919485883,2537512,0.002333,2025-07-15
2025-07-15 00:07:23+00:00,0xfbc2005b8af6b103f80d373a7c1dcc9af5a9c43f9ebf...,Liquidator (rewards),rewards,3950519048,1873497,0.007401,2025-07-15
2025-07-14 23:53:59+00:00,0xa8f76c5b99ad5034f9fca0e7bd62c404cdfb27c69eb4...,Custom Oracle Executor (pricing),pricing,2571813534,40641,0.000105,2025-07-08


In [ ]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# 1. Make sure your datetime is really a datetime and pull out the date
df2 = df2.copy()
df2["date"] = pd.to_datetime(df2.index.date)

# 2. Aggregate total gas_used and total fee (gas_cost_in_eth) per day & category
daily = (
    df2.groupby(["date", "category"])
    .agg(gas_used=("gas_used", "sum"), fee_eth=("gas_cost_in_eth", "sum"))
    .reset_index()
)

# 3. Compute daily totals and %‐share by category
totals = daily.groupby("date")[["gas_used", "fee_eth"]].sum().rename(columns=lambda c: "total_" + c)
daily = daily.merge(totals, on="date")
daily["pct_gas_used"] = daily["gas_used"] / daily["total_gas_used"] * 100
daily["pct_fee_eth"] = daily["fee_eth"] / daily["total_fee_eth"] * 100

# 4. Pivot to wide form for plotting
gas_pct = daily.pivot(index="date", columns="category", values="pct_gas_used").fillna(0)
fee_pct = daily.pivot(index="date", columns="category", values="pct_fee_eth").fillna(0)

px.bar(gas_pct, title="Gas Used Percent by Category")

In [35]:
df["date"] = pd.to_datetime(df["datetime"]).dt.date

# get only july 15 2025 rows
# and cateogry is keeper_network


suspect_df = df[(df["date"] == pd.to_datetime("2025-07-15").date()) & (df["category"] == "keeper_network")].sort_values(
    by="datetime"
)
suspect_df

,datetime,tx_hash,label,category,effective_gas_price,gas_used,gas_cost_in_eth,week_start,date
19391,2025-07-15 00:00:23+00:00,0x0932d313988911f9c96d476d1d7cd6af6c7e2b457da7...,Calculators (keeper_network),keeper_network,1012155154,2036895,0.002474,2025-07-15,2025-07-15
19407,2025-07-15 00:00:59+00:00,0x66182f8683ea89cdaeb7f673fedfbc3a0884466955df...,Calculators (keeper_network),keeper_network,1016957158,1754736,0.002141,2025-07-15,2025-07-15
19277,2025-07-15 00:01:35+00:00,0x1758510676401cfe3f8753c5903d4b69f5ffee6beb47...,Calculators (keeper_network),keeper_network,1977487865,1626555,0.003860,2025-07-15,2025-07-15
19395,2025-07-15 00:02:11+00:00,0x279ffaf85b987d30209d5da38746fb0ff3eaad37c940...,Calculators (keeper_network),keeper_network,1514899941,1202970,0.002187,2025-07-15,2025-07-15
19405,2025-07-15 01:00:35+00:00,0xf1debe333e6d29b9abbd5b9bf102a0f0efe990e0b9c3...,Calculators (keeper_network),keeper_network,1875633359,3883365,0.008741,2025-07-15,2025-07-15
...,...,...,...,...,...,...,...,...,...
19151,2025-07-15 14:00:47+00:00,0x37e1753bf3cba1586665b8fd5b5dee4af15523ca206b...,Calculators (keeper_network),keeper_network,6384563106,652353,0.004998,2025-07-15,2025-07-15
19232,2025-07-15 14:19:35+00:00,0x4f8b8a15749fc111feed1d4f119feb36fd6e63dfddfd...,Eth Per Token Sender (keeper_network),keeper_network,4898790007,333030,0.001958,2025-07-15,2025-07-15
19251,2025-07-15 14:55:59+00:00,0xaaeeef638b8672fa08c5ff611c431187672ccdf19239...,Incentive Pricing (keeper_network),keeper_network,15744489474,755116,0.014267,2025-07-15,2025-07-15
19168,2025-07-15 15:00:59+00:00,0x797499b5d5411eb318b02f1450d06d2e0cc355a31372...,Incentive Pricing (keeper_network),keeper_network,15734051111,214757,0.004055,2025-07-15,2025-07-15


In [36]:
px.scatter(
    suspect_df,
    x="datetime",
    y="effective_gas_price",
    color="category",
    title="Scatter plot of gas price x time for July 15 keeper network transactions",
)

In [27]:
px.bar(fee_pct, title="Gas Cost in ETH Percent by Category")

In [ ]:
def _hourly_box_plot_of_gas_prices(our_upkeep_df: pd.DataFrame):
    # Group by hour of the day and aggregate gas prices
    hourly_gas_price = our_upkeep_df.groupby(our_upkeep_df.index.hour)["gas_price"]
    hourly_gas_price_df = hourly_gas_price.apply(list).reset_index()
    hourly_gas_price_df.columns = ["Hour", "GasPrices"]

    # Explode the gas prices into individual rows
    exploded_df = hourly_gas_price_df.explode("GasPrices")
    exploded_df["GasPrices"] = exploded_df["GasPrices"].astype(float)

    # Create the box plot for hourly distribution of gas prices
    hourly_gas_price_box_and_whisker_fig = px.box(
        exploded_df, x="Hour", y="GasPrices", title="UTC, Hourly Distribution of Gas Prices"
    )

    return hourly_gas_price_box_and_whisker_fig

In [ ]:
break

In [ ]:
import streamlit as st

for col in ["category", "label"]:
    st.subheader(f"Gas Costs in ETH and Gas Used by Expense {col}")
    for value_col in ["gas_cost_in_eth", "gas_used"]:
        daily_sum_df = (
            df.groupby([col, "datetime"])[value_col]
            .sum()
            .reset_index()
            .pivot(index="datetime", columns=col, values=value_col)
            .fillna(0)
            .resample("1D")
            .sum()
        )

        st.subheader(f"{value_col} by {col}")
        for n_days in [1, 7, 30]:
            fig = px.bar(
                daily_sum_df.rolling(window=n_days).sum(),
                title=f"{n_days}-day rolling sum Mainnet {value_col}",
            )
            fig.update_yaxes(title_text=value_col)
            st.plotly_chart(fig, use_container_width=True)

In [6]:
# Ensure datetime column is proper and extract the date
df["date"] = pd.to_datetime(df["datetime"]).dt.date

# Create weighted value column
df["gas_price_times_gas_used"] = df["effective_gas_price"] * df["gas_used"] / 1e9

# Group and compute sum of weights and weighted values
grouped = df.groupby(["date", "category"]).agg(
    weighted_sum=("gas_price_times_gas_used", "sum"), total_weight=("gas_used", "sum")
)

# Compute weighted average
grouped["weighted_avg_effective_gas_price"] = grouped["weighted_sum"] / grouped["total_weight"]

# Optionally drop intermediate columns
grouped = (
    grouped.drop(columns=["weighted_sum", "total_weight"])
    .reset_index()
    .pivot(index="date", columns="category", values="weighted_avg_effective_gas_price")
)

px.line(
    grouped,
    title="Weighted Average Effective Gas Price by Date and EOA Address",
    labels={"value": "Weighted Average Effective Gas Price", "date": "Date"},
).update_yaxes(title_text="Effective Gas Price (Gwei)").show()

In [14]:
import pandas as pd
import plotly.express as px

N = 30  # <-- your rolling window size in days

# 1. Parse and index
# df["datetime"] = pd.to_datetime(df["datetime"])
# df = df.set_index("datetime").sort_index()

# 2. Compute per-transaction contributions
df["weighted_contrib"] = df["effective_gas_price"] * df["gas_used"] / 1e9
df["weight"] = df["gas_used"]

# 3. Aggregate to daily sums per category
#    This gives you a MultiIndex: (category, datetime)
daily = (
    df.groupby("category").resample("D").agg(weighted_sum=("weighted_contrib", "sum"), total_weight=("weight", "sum"))
)

# 4. Pivot so each category is its own column
ws = daily["weighted_sum"].unstack("category").fillna(0)
tw = daily["total_weight"].unstack("category").fillna(0)

# 5. Compute N‑day rolling sums
ws_roll = ws.rolling(window=N, min_periods=1).sum()
tw_roll = tw.rolling(window=N, min_periods=1).sum()

# 6. Divide to get the rolling weighted avg
wavg_roll = ws_roll / tw_roll

# 7. Plot
fig = px.line(
    wavg_roll,
    title=f"{N}-Day Rolling Weighted‑Avg Effective Gas Price by Category",
    labels={"value": "Weighted‑Avg Effective Gas Price", "datetime": "Date"},
)
fig.update_yaxes(title_text="Effective Gas Price (Gwei)")
fig.show()

In [17]:
df.drop_duplicates()

,label,category,effective_gas_price,gas_used,gas_cost_in_eth,gas_price_times_gas_used,weighted_contrib,weight
datetime,,,,,,,,
2024-08-30 16:18:35+00:00,Custom Oracle Executor (pricing),pricing,4318952717,57192,0.000247,2.470095e+05,2.470095e+05,57192
2024-08-30 18:33:35+00:00,Custom Oracle Executor (pricing),pricing,4012396541,40629,0.000163,1.630197e+05,1.630197e+05,40629
2024-08-30 19:52:35+00:00,Custom Oracle Executor (pricing),pricing,2629695373,40629,0.000107,1.068419e+05,1.068419e+05,40629
2024-08-30 20:39:35+00:00,Custom Oracle Executor (pricing),pricing,2555462516,40653,0.000104,1.038872e+05,1.038872e+05,40653
2024-08-31 00:57:35+00:00,Custom Oracle Executor (pricing),pricing,2224735824,40653,0.000090,9.044219e+04,9.044219e+04,40653
...,...,...,...,...,...,...,...,...
2025-07-15 14:00:47+00:00,Calculators (keeper_network),keeper_network,6384563106,652353,0.004998,4.164989e+06,4.164989e+06,652353
2025-07-15 14:19:35+00:00,Eth Per Token Sender (keeper_network),keeper_network,4898790007,333030,0.001958,1.631444e+06,1.631444e+06,333030
2025-07-15 14:55:59+00:00,Incentive Pricing (keeper_network),keeper_network,15744489474,755116,0.014267,1.188892e+07,1.188892e+07,755116
